In [33]:
import os
import sys
import json
import itertools

import pandas as pd

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(os.path.join(module_path))

from tensorflow import keras
from scipy.stats import shapiro, f_oneway, kruskal
from statsmodels.stats.multicomp import MultiComparison

from functions.plotting import plot_bars, plot_bar_from_dict

WIDTH = 150
HEIGHT = 150

In [2]:
data_dir = os.path.join(
    '..',
    '..',
    '..',
    'data1',
    'images_original_inception_resnet_v2_150x150_categorized')
test_data_dir = os.path.join(data_dir, 'validation')
test_ds = keras.utils.image_dataset_from_directory(
    directory=test_data_dir,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(HEIGHT, WIDTH))

Found 2000 files belonging to 9 classes.


In [3]:
models_dir = os.path.join(
    '..',
    '..',
    'models',
    'data1')
model_names = os.listdir(models_dir)
non_normalized_models = \
    list(
        filter(
            lambda model_file_name: not 'normalization' in model_file_name,
            model_names))

In [4]:
def get_accuracies_and_losses(models: list[str]) -> (dict[str, float], dict[str, float]):
    accuracies = {}
    losses = {}

    for model_name in models:
        model_path = os.path.join(models_dir, model_name)
        model = keras.models.load_model(model_path)
        loss, accuracy = model.evaluate(test_ds)
        accuracies[model_name] = accuracy
        losses[model_name] = loss

    return accuracies, losses

In [5]:
accuracies_wo_norm_data_path = os.path.join('..', '..', 'histories', 'accuracies_wo_norm.json')
losses_wo_norm_data_path = os.path.join('..', '..', 'histories', 'losses_wo_norm.json')

In [6]:
if not os.path.exists(accuracies_wo_norm_data_path):
    accuracies_wo_norm, losses_wo_norm = get_accuracies_and_losses(non_normalized_models)

    with open(accuracies_wo_norm_data_path, 'w') as json_file:
        json.dump(accuracies_wo_norm, json_file, indent=4)
    with open(losses_wo_norm_data_path, 'w') as json_file:
        json.dump(losses_wo_norm, json_file, indent=4)

In [7]:
with open(accuracies_wo_norm_data_path, 'r') as losses_file:
    accuracies_wo_norm = json.load(losses_file)
with open(losses_wo_norm_data_path, 'r') as losses_file:
    losses_wo_norm = json.load(losses_file)

In [ ]:
root = os.path.join('..', '..', 'plots', 'testing')
accuracies_path = os.path.join(root, 'accuracies.pdf')

plot_bar_from_dict(accuracies_wo_norm, 'Model Accuracies', 'Name', 'Accuracy', accuracies_path)

In [ ]:
losses_path = os.path.join(root, 'losses.pdf')

plot_bar_from_dict(losses_wo_norm, 'Model Losses', 'Name', 'Accuracy', losses_path)

In [8]:
normalized_models = \
    list(
    filter(
        lambda model_file_name: 'normalization' in model_file_name,
        model_names))

In [ ]:
accuracies_w_norm, losses_w_norm = get_accuracies_and_losses(normalized_models)

In [8]:
accuracies_norm_data_path = os.path.join('..', '..', 'histories', 'accuracies_norm.json')
losses_norm_data_path = os.path.join('..', '..', 'histories', 'losses_norm.json')

In [9]:
if not os.path.exists(accuracies_norm_data_path):
    accuracies_norm, losses_norm = get_accuracies_and_losses(normalized_models)

    with open(accuracies_norm_data_path, 'w') as json_file:
        json.dump(accuracies_norm, json_file, indent=4)
    with open(losses_norm_data_path, 'w') as json_file:
        json.dump(losses_norm, json_file, indent=4)

In [10]:
with open(accuracies_norm_data_path, 'r') as losses_file:
    accuracies_norm = json.load(losses_file)
with open(losses_norm_data_path, 'r') as losses_file:
    losses_norm = json.load(losses_file)

In [ ]:
accuracies_path = os.path.join(root, 'accuracies_norm.pdf')

plot_bar_from_dict(accuracies_norm, 'Model Accuracies', 'Name', 'Accuracy', accuracies_path)

In [ ]:
losses_path = os.path.join(root, 'losses_norm.pdf')

plot_bar_from_dict(losses_norm, 'Model Losses', 'Name', 'Accuracy', losses_path)

Analysis part of model testing

In [11]:
model_categories = set(map(lambda name: '_'.join(name.split('_')[:-2]), model_names))
grouped_models = {key: list(filter(lambda name: name.startswith(key), model_names)) for key in model_categories}
keyed_accuracies_wo_norm = {key: [] for key in grouped_models if 'normalization' not in key}
keyed_accuracies_norm = {key: [] for key in grouped_models if 'normalization' in key}
keyed_losses_wo_norm = {key: [] for key in grouped_models if 'normalization' not in key}
keyed_losses_norm = {key: [] for key in grouped_models if 'normalization' in key}


def categorize_numbers(for_dict: dict[str, list[float]], numbers_dict: dict[str, float]) -> None:
    for group, value in for_dict.items():
        vals = [
            value for key, value in numbers_dict.items()
            if key.startswith(group)]

        value.extend(vals)


categorize_numbers(keyed_accuracies_norm, accuracies_norm)
categorize_numbers(keyed_accuracies_wo_norm, accuracies_wo_norm)
categorize_numbers(keyed_losses_norm, losses_norm)
categorize_numbers(keyed_losses_wo_norm, losses_wo_norm)

In [12]:
accuracies_from_norm_models = pd.DataFrame(keyed_accuracies_norm)
accuracies_from_no_norm_models = pd.DataFrame(keyed_accuracies_wo_norm)
losses_from_norm_models = pd.DataFrame(keyed_losses_norm)
losses_from_no_norm_models = pd.DataFrame(keyed_losses_wo_norm)

In [13]:
accuracies_from_norm_models.describe()

,inception_ignoring_imbalance_4_only_bottom_half_layers_trainable_with_attention_module_on_top_and_samplewise_normalization,inception_ignoring_imbalance_3_whole_model_trainable_with_attention_module_on_top_and_samplewise_normalization,inception_ignoring_imbalance_1_whole_model_trainable_samplewise_normalization,inception_ignoring_imbalance_2_only_bottom_half_layers_trainable_samplewise_normalization
count,30.000000,30.000000,30.000000,30.000000
mean,0.759717,0.753100,0.748417,0.752600
std,0.011242,0.018877,0.016311,0.014671
min,0.739000,0.700500,0.705000,0.714000
25%,0.752125,0.746750,0.741000,0.747000
50%,0.759250,0.758000,0.753250,0.754750
75%,0.766375,0.765250,0.760500,0.762000
max,0.778500,0.783000,0.769000,0.782000


In [14]:
accuracies_from_no_norm_models.describe()

,inception_ignoring_imbalance_1_whole_model_trainable,inception_ignoring_imbalance_2_only_bottom_half_layers_trainable,inception_ignoring_imbalance_3_whole_model_trainable_with_attention_module_on_top,inception_ignoring_imbalance_4_only_bottom_half_layers_trainable_with_attention_module_on_top
count,30.000000,30.000000,30.000000,30.000000
mean,0.774250,0.792350,0.793567,0.792917
std,0.016210,0.013289,0.010632,0.007862
min,0.733500,0.762000,0.777000,0.777500
25%,0.769125,0.784375,0.782750,0.788750
50%,0.776500,0.794250,0.795500,0.794250
75%,0.784875,0.801750,0.799750,0.797375
max,0.801500,0.813500,0.816000,0.805000


In [15]:
losses_from_norm_models.describe()

,inception_ignoring_imbalance_4_only_bottom_half_layers_trainable_with_attention_module_on_top_and_samplewise_normalization,inception_ignoring_imbalance_3_whole_model_trainable_with_attention_module_on_top_and_samplewise_normalization,inception_ignoring_imbalance_1_whole_model_trainable_samplewise_normalization,inception_ignoring_imbalance_2_only_bottom_half_layers_trainable_samplewise_normalization
count,30.000000,30.000000,30.000000,30.000000
mean,0.673036,0.691220,0.709701,0.703292
std,0.026645,0.048443,0.052602,0.044011
min,0.629656,0.641208,0.636466,0.652724
25%,0.650746,0.659710,0.674715,0.676151
50%,0.670872,0.675271,0.692928,0.699332
75%,0.694725,0.706006,0.743005,0.722880
max,0.722168,0.837553,0.825080,0.869296


In [16]:
losses_from_no_norm_models.describe()

,inception_ignoring_imbalance_1_whole_model_trainable,inception_ignoring_imbalance_2_only_bottom_half_layers_trainable,inception_ignoring_imbalance_3_whole_model_trainable_with_attention_module_on_top,inception_ignoring_imbalance_4_only_bottom_half_layers_trainable_with_attention_module_on_top
count,30.000000,30.000000,30.000000,30.000000
mean,0.628121,0.596638,0.587635,0.582946
std,0.046395,0.033383,0.022151,0.018216
min,0.572206,0.541605,0.538928,0.552645
25%,0.596247,0.575664,0.571949,0.572053
50%,0.621229,0.590225,0.584879,0.578076
75%,0.644392,0.625841,0.601979,0.592483
max,0.747070,0.668844,0.633576,0.641276


In [17]:
ALPHA = .05
anova_testable_groups = []
kruskal_walis_testable_groups = []


def assign_models_to_groups(models: pd.DataFrame) -> None:
    for col in models.columns:
        _, p_value = shapiro(models[col])

        if p_value >= ALPHA:
            anova_testable_groups.append(col)
        else:
            kruskal_walis_testable_groups.append(col)


assign_models_to_groups(accuracies_from_norm_models)
assign_models_to_groups(accuracies_from_no_norm_models)

In [30]:
anova_values = []
kruskal_values = []
anova_cols = []

for col in anova_testable_groups:
    if col in accuracies_from_norm_models:
        anova_values.append(accuracies_from_norm_models[col].tolist())
        anova_cols.append(col)
    if col in accuracies_from_no_norm_models:
        anova_values.append(accuracies_from_no_norm_models[col].tolist())
        anova_cols.append(col)

for col in kruskal_walis_testable_groups:
    if col in accuracies_from_norm_models:
        kruskal_values.append(accuracies_from_norm_models[col].tolist())
    if col in accuracies_from_no_norm_models:
        kruskal_values.append(accuracies_from_no_norm_models[col].tolist())


f_stat_anova, p_value_anova = f_oneway(*anova_values)
h_stat_kruskal, p_value_kruskal = kruskal(*kruskal_values)

print(f'ANOVA p-value: {p_value_anova}')
print(f'Kruskal-Wallis p-value: {p_value_kruskal}')

ANOVA p-value: 4.064446737409379e-37
Kruskal-Wallis p-value: 0.17359764347681522


ANOVA p-value is extremely small, practically zero, which strongly indicates that there are significant differences between the means of the groups tested (those groups which passed the normality test). I reject the null hypothesis of equal means. Given this result, it's warranted to proceed to post-hoc analysis (like Tukey's HSD) to find out which specific groups have significantly different means.

Kruskal-Wallis p-value is relatively large (greater than common thresholds like 0.05 or 0.01), which suggests that I fail to reject the null hypothesis for the Kruskal-Wallis test. This indicates that there isn't a statistically significant difference between the median ranks of the groups tested (those groups which failed the normality test).

In [41]:
tukey_vals = list(itertools.chain(*anova_values))
tukey_cols = []
idx = 0

for col in anova_values:
    n_repeat = len(col)
    group = [anova_cols[idx]] * n_repeat
    idx += 1

    tukey_cols.extend(group)

mc = MultiComparison(tukey_vals, tukey_cols)
result = str(mc.tukeyhsd())

with open(os.path.join('..', '..', 'debug_data', 'tukeyhsd.test'), 'w') as test_file:
    test_file.write(result)